# Cancer Alpha: Data Preprocessing Pipeline

This notebook demonstrates the complete data preprocessing pipeline for the Cancer Alpha multi-cancer classification system.

## Overview
- Load TCGA genomic and clinical data
- Feature engineering and selection
- Data quality assessment
- Preprocessing for machine learning

## Citation
If you use this code, please cite:
**Cancer Alpha: A Production-Ready AI System for Multi-Cancer Classification Achieving 95% Balanced Accuracy on Real TCGA Data**

In [ ]:
import numpy as np
import pandas as pd
import pickle
import json
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import RobustScaler, LabelEncoder
from sklearn.feature_selection import mutual_info_classif
from sklearn.impute import KNNImputer
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8')

print("Cancer Alpha Data Preprocessing Pipeline")
print("=======================================")

In [ ]:
# Load TCGA data
data_path = Path('../data/tcga')

print("Loading TCGA data...")
tcga_mutation = np.load(data_path / 'tcga_mutation.npy')
tcga_clinical = np.load(data_path / 'tcga_clinical.npy')
tcga_labels = np.load(data_path / 'tcga_labels.npy')

print(f"Mutation data shape: {tcga_mutation.shape}")
print(f"Clinical data shape: {tcga_clinical.shape}")
print(f"Labels shape: {tcga_labels.shape}")

# Cancer types mapping
cancer_types = {
    0: 'BRCA', 1: 'LUAD', 2: 'COAD', 3: 'PRAD',
    4: 'STAD', 5: 'KIRC', 6: 'HNSC', 7: 'LIHC'
}

# Convert to DataFrame for easier manipulation
n_samples = tcga_mutation.shape[0]
mutation_cols = [f'gene_{i}' for i in range(tcga_mutation.shape[1])]
clinical_cols = [f'clinical_{i}' for i in range(tcga_clinical.shape[1])]

df_mutation = pd.DataFrame(tcga_mutation, columns=mutation_cols)
df_clinical = pd.DataFrame(tcga_clinical, columns=clinical_cols)
df_labels = pd.DataFrame(tcga_labels, columns=['cancer_type'])

# Combine all features
df_combined = pd.concat([df_mutation, df_clinical, df_labels], axis=1)

print(f"\nCombined dataset shape: {df_combined.shape}")
print(f"Cancer type distribution:")
for i, cancer in cancer_types.items():
    count = (df_combined['cancer_type'] == i).sum()
    print(f"  {cancer}: {count} samples")

In [ ]:
# Feature Engineering
print("Performing feature engineering...")

# Create mutation burden features
df_combined['total_mutations'] = df_mutation.sum(axis=1)
df_combined['mutation_rate'] = df_combined['total_mutations'] / df_mutation.shape[1]
df_combined['unique_genes_mutated'] = (df_mutation > 0).sum(axis=1)

# Key cancer genes (based on manuscript)
key_genes = ['TP53', 'KRAS', 'PIK3CA', 'APC', 'EGFR', 'BRCA1', 'BRCA2']
# Simulate key gene positions (in real implementation, these would be mapped)
key_gene_positions = [0, 15, 23, 41, 67, 89, 90]  # Example positions

for i, gene in enumerate(key_genes):
    if key_gene_positions[i] < df_mutation.shape[1]:
        df_combined[f'{gene}_mutation'] = df_mutation.iloc[:, key_gene_positions[i]]

print(f"Added {len(key_genes)} key cancer gene features")
print(f"Total features after engineering: {df_combined.shape[1]}")

In [ ]:
# Feature Selection using Mutual Information
print("Performing feature selection...")

# Separate features and target
X = df_combined.drop(['cancer_type'], axis=1)
y = df_combined['cancer_type']

# Handle missing values with KNN imputation
imputer = KNNImputer(n_neighbors=5)
X_imputed = imputer.fit_transform(X)
X_imputed = pd.DataFrame(X_imputed, columns=X.columns)

# Calculate mutual information scores
mi_scores = mutual_info_classif(X_imputed, y, random_state=42)
mi_scores = pd.Series(mi_scores, index=X.columns).sort_values(ascending=False)

# Select top 150 features (as mentioned in manuscript)
top_150_features = mi_scores.head(150).index.tolist()

print(f"Selected top 150 features from {len(mi_scores)} total features")
print(f"\nTop 10 features by mutual information:")
for i, (feature, score) in enumerate(mi_scores.head(10).items()):
    print(f"{i+1:2d}. {feature}: {score:.4f}")

# Create final feature set
X_selected = X_imputed[top_150_features]
print(f"\nFinal feature matrix shape: {X_selected.shape}")

In [ ]:
# Data Scaling
print("Scaling features using RobustScaler...")

scaler = RobustScaler()
X_scaled = scaler.fit_transform(X_selected)
X_scaled = pd.DataFrame(X_scaled, columns=X_selected.columns)

# Label encoding
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

print(f"Scaled feature matrix shape: {X_scaled.shape}")
print(f"Label distribution after encoding:")
unique, counts = np.unique(y_encoded, return_counts=True)
for label, count in zip(unique, counts):
    cancer_name = cancer_types[label]
    print(f"  {label} ({cancer_name}): {count} samples")

In [ ]:
# Save processed data and preprocessing objects
print("Saving processed data...")

output_path = Path('../models')
output_path.mkdir(exist_ok=True)

# Save processed data
X_scaled.to_csv(output_path / 'X_processed.csv', index=False)
pd.DataFrame(y_encoded, columns=['cancer_type']).to_csv(output_path / 'y_processed.csv', index=False)

# Save preprocessing objects
with open(output_path / 'imputer.pkl', 'wb') as f:
    pickle.dump(imputer, f)

with open(output_path / 'scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

with open(output_path / 'label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)

# Save feature names and metadata
metadata = {
    'n_samples': int(X_scaled.shape[0]),
    'n_features': int(X_scaled.shape[1]),
    'feature_names': top_150_features,
    'cancer_types': cancer_types,
    'preprocessing_steps': [
        'KNN Imputation (k=5)',
        'Mutual Information Feature Selection (top 150)',
        'Robust Scaling',
        'Label Encoding'
    ]
}

with open(output_path / 'preprocessing_metadata.json', 'w') as f:
    json.dump(metadata, f, indent=2)

print("Preprocessing complete!")
print(f"Processed data saved to: {output_path}")
print(f"Final dataset: {X_scaled.shape[0]} samples, {X_scaled.shape[1]} features, {len(np.unique(y_encoded))} cancer types")

In [ ]:
# Data Visualization
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Cancer type distribution
cancer_counts = pd.Series(y_encoded).map(cancer_types).value_counts()
axes[0, 0].bar(cancer_counts.index, cancer_counts.values)
axes[0, 0].set_title('Cancer Type Distribution')
axes[0, 0].set_ylabel('Number of Samples')
axes[0, 0].tick_params(axis='x', rotation=45)

# Feature importance (top 20)
top_20_mi = mi_scores.head(20)
axes[0, 1].barh(range(len(top_20_mi)), top_20_mi.values)
axes[0, 1].set_yticks(range(len(top_20_mi)))
axes[0, 1].set_yticklabels(top_20_mi.index)
axes[0, 1].set_title('Top 20 Features by Mutual Information')
axes[0, 1].set_xlabel('Mutual Information Score')

# Mutation burden distribution
mutation_burden = df_combined['total_mutations']
axes[1, 0].hist(mutation_burden, bins=30, alpha=0.7)
axes[1, 0].set_title('Distribution of Mutation Burden')
axes[1, 0].set_xlabel('Total Mutations per Sample')
axes[1, 0].set_ylabel('Frequency')

# Missing data heatmap (before imputation)
missing_data = X.isnull().sum().head(50)
axes[1, 1].bar(range(len(missing_data)), missing_data.values)
axes[1, 1].set_title('Missing Data (First 50 Features)')
axes[1, 1].set_xlabel('Feature Index')
axes[1, 1].set_ylabel('Missing Values Count')

plt.tight_layout()
plt.savefig(output_path / 'preprocessing_summary.png', dpi=300, bbox_inches='tight')
plt.show()

print("Data preprocessing visualization complete!")